<a href="https://colab.research.google.com/github/dvschultz/MUNIT/blob/master/MUNIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MUNIT
MUNIT stands for Multimodal UNsupervised Image-to-image Translation. That’s a lot of words to say it can convert images of cats to images of dogs, or images of horses to images of zebras. Basically, it learns to convert one set of images (a “domain”) to another set.

In [2]:
!git clone https://github.com/dvschultz/MUNIT

#install dependencies
!pip install torch torchvision tensorboard tensorboardX PyYAML torchfile
%cd MUNIT

Cloning into 'MUNIT'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 444 (delta 13), reused 0 (delta 0), pack-reused 418
Receiving objects: 100% (444/444), 6.12 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (225/225), done.
     |████████████████████████████████| 204kB 8.8MB/s 
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=26b570ab36dc8ed307fa119decbff48427f70387bfd57f47196a52aabd569089
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile
/content/MUNIT


#Dataset Preparation
_Personally, I would not do training of MUNIT on Colab. Colab is slow and can quit after a couple hours—you’ll need more time than that to properly train a MUNIT model. I recommend training elsewhere and then jumping down to the Testing section here in Colab to test your model._

MUNIT requires two different sets of images. Unlike Pix2Pix or similar models, these two sets of images do not need to be tightly coupled (for example: a color image and its grayscale equivalent). The images also do not need to be the same size.

I recommend a minimum of ~250 images per folder. This will allow for the model to generalize fairly well (the more images, the better). 

Once you have created the images you will need to upload them to your server. The folder structure should look like this:
* Top level folder name (I usually use X2Y, where X is the first dataset and Y is the second.)
* ├ TrainA (training set from X)
* ├ TrainB (training set from Y)
* ├ TestA (testing set from X)
* ├ TestB (testing set from Y)

For our purposes TrainA/TestA and TrainB/TestB can be the same, but in a "scientific" setting you would want these folder to contain different images.

There are two ways to upload the images.
1. Zip up the folder and upload it to Colab in the `MUNIT/datasets` folder. This is probably slow and depending how large your dataset is can use up a lot of the Disk space in Colab.
2. You can connect your Google Drive to Colab and host your files in Google Drive (This does not require you zipping up your folder, but does require a Google Drive account). This option needs a little setting up, but it saves you disk space and time once it’s set up.


In [0]:
#Option 2: Connect your Google Drive to Colab.
from google.colab import drive
drive.mount('/content/gdrive')

#after running this command you will need to click on a URL and copy the authorization code. Then come back here and pastr it in the field below

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


You could now have access to your Google Drive in the Files tab to the left of this text (Click on the left pointing error if your panel is collapsed.)

#Training
_I repeat: I do not recommend training on Colab. But here are the instructions if you know what you’re doing and really must._

Once you have your dataset prepped and uploaded, the last thing to do is create a YAML config file. Let’s first duplicate one of the example files. In the code below I recommend renaming the file to match the top level folder name you used when uploading the dataset. This helps keep track of file naming and will remind you what settings you used should you need to retrain the model ever again.

In [2]:
pwd

'/content/MUNIT'

#Testing
If you’re only using Colab for testing, you’ll need to do the following:
1. Move your config .yaml from training into the `configs` folder.
2. Create a `models` folder and put the generator .pt file in there (the generator file begins with `gen_`)
3. Create an `inputs` folder and a `styles` folder. You’ll put images you want to translate in the `inputs` folder and images you want to guide style in the `styles` folder (See the Guided Translation).

In [0]:
#make some folders (only run once)
%mkdir models
%mkdir styles

In [0]:
#testing with a single image
#this generates 10 random images from the model
!python test.py --config /content/MUNIT/configs/512-sm_birds2floralmag_folder.yaml --input /content/MUNIT/inputs/birdsAustraliav5Goul_0208.png --output_folder outputs --checkpoint /content/MUNIT/models/gen_00150000.pt --a2b 1

In [17]:
!zip -r outputs.zip ./outputs

updating: outputs/ (stored 0%)
updating: outputs/output004.jpg (deflated 1%)
updating: outputs/output006.jpg (deflated 3%)
updating: outputs/output002.jpg (deflated 1%)
updating: outputs/output009.jpg (deflated 1%)
updating: outputs/output007.jpg (deflated 1%)
updating: outputs/output005.jpg (deflated 2%)
updating: outputs/output000.jpg (deflated 3%)
updating: outputs/output001.jpg (deflated 1%)
updating: outputs/output003.jpg (deflated 1%)
updating: outputs/output008.jpg (deflated 0%)
updating: outputs/input.jpg (deflated 1%)
updating: outputs/output069.jpg (deflated 2%)
updating: outputs/output077.jpg (deflated 2%)
updating: outputs/output091.jpg (deflated 1%)
updating: outputs/output037.jpg (deflated 1%)
updating: outputs/output032.jpg (deflated 2%)
updating: outputs/output065.jpg (deflated 3%)
updating: outputs/output087.jpg (deflated 1%)
updating: outputs/output013.jpg (deflated 1%)
updating: outputs/output088.jpg (deflated 1%)
updating: outputs/output040.jpg (deflated 1%)
updatin

##Testing Options

`--num_style`
How many "styles" do you want to output

In [0]:
#testing with a single image, use --num_styles to set the number of styles
!python test.py --config /content/MUNIT/configs/512-sm_birds2floralmag_folder.yaml --input /content/MUNIT/inputs/birdsAustraliav5Goul_0212.png --output_folder outputs --checkpoint /content/MUNIT/models/gen_00150000.pt --a2b 1 --num_style 100

##Guided Translation
MUNIT also allows you to do what they call "guided translation." This allows you to pass it an image and use that image as the "style" (for lack of a better term) for the output image.

In [0]:
#guided translation
python test.py --config configs/edges2shoes_folder.yaml --input inputs/edges2shoes_edge.jpg --output_folder results --checkpoint models/edges2shoes.pt --a2b 1 --style inputs/edges2shoes_shoe.jpg

##Batch Testing
Both of the above examples only generate samples from one image. If you want to test it on a whole folder of images you use the `test_batch.py` script.

Note: you can’t currently batch test with a guided translation image. (I’m working on it.)

In [0]:
python test_batch.py --config configs/edges2shoes_folder.yaml --input_folder inputs --output_folder outputs --checkpoint models/edges2shoes.pt --a2b 1